# CASO AMAZON 

Integrantes:

    -Hernan Carmelo Cavadias Martinez
    -Duvan Santiago Castro Bolivar 
    -David Santiago Buitrago Norato
    -Miguel Angel Malagón Romero

## Data Understanding

## Data Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import platform
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
# Importacion de la base de datos
datos = pd.read_excel('DB.xlsx')

In [45]:
datos.describe().T

,count,mean,std,min,25%,50%,75%,max
uid,1600.0,2948.575000,1717.270229,5.0,1478.5,2877.0,4395.5,6087.0
satis,1600.0,7.575625,1.192233,2.0,7.0,8.0,8.0,10.0
poverq,1600.0,7.573750,1.362248,2.0,7.0,8.0,8.0,10.0
soverq,1600.0,7.548750,1.314802,2.0,7.0,8.0,8.0,10.0
pq,1600.0,7.573125,1.363200,2.0,7.0,8.0,8.0,10.0
VN_1009_Q20A,1600.0,7.669375,1.243897,2.0,7.0,8.0,8.0,10.0
VN_1009_TP01,1600.0,7.858125,1.046987,3.0,7.0,8.0,9.0,10.0
VN_1009_TP02,1600.0,7.596250,1.267573,2.0,7.0,8.0,9.0,10.0
VN_1009_TP03,1600.0,7.631875,1.199900,2.0,7.0,8.0,8.0,10.0
VN_1009_TP04,1600.0,7.742500,1.179327,2.0,7.0,8.0,9.0,10.0


Como podemos ver la mayoría de las variables son categóricas las cuales se encuentran entre 1 y 10, notamos que en algunas variables tenemos observaciones que registran un 97, direccionándonos al diccionario de la base de datos, notamos que hace parte de las observaciones en donde los encuetados no saben/ no responden. 

Para tratar dichas observaciones vamos a hacer un conteo del total de observaciones que presentan dichas problemáticas, para analizar el eliminarlas o no del modelo. 
 

In [4]:
(datos != 97).all(axis=1).value_counts()

True     1311
False     289
dtype: int64

Como vemos notamos que hay 289 observaciones que presentan estas problematicas, con el fin de no afectar nuestro modelo lo vamos a borrar. 

In [46]:
filtro = (datos[datos.columns[5:]]!=97)
datos2 = datos[filtro]
datos2.update(datos.iloc[:, 0:5])
datos2.dropna(axis=0,inplace=True)
datos2.reset_index(drop=True, inplace=True)
datos2.shape

(1311, 22)

Corregido el error, vamos volver categorica la variable de étinia a través de Dummis.

In [47]:
etnia = {1:"Chinese", 2:"Malay", 3:"Indian", 4:"Eurasian", 5:"Others"}
for valor, nv in etnia.items():
    datos2['race'] = datos2['race'].replace(valor, nv)

In [48]:
df_dummi = pd.get_dummies(datos2['race'],prefix_sep ='_',drop_first=True)
datos2 = pd.concat([datos2, df_dummi], axis=1)
datos2.rename(columns={'race':"Chinese"}, inplace=True)
datos2['Chinese'] = datos2['Chinese'].apply(lambda x: 1 if x == 'Chinese' else 0) # remplaza por 1 y 0
datos2['VN_1009_TP25A'] = datos2['VN_1009_TP25A'].apply(lambda x: 1 if x == 2 else 0) # Tipo de tienda (virtual)
datos2.head(3)

,uid,company_v,satis,poverq,soverq,pq,VN_1009_Q20A,VN_1009_TP01,VN_1009_TP02,VN_1009_TP03,...,VN_1009_TP16,VN_1009_TP17,VN_1009_TP19,VN_1009_TP25A,age,Chinese,Eurasian,Indian,Malay,Others
0,5.0,ZALORA,7.0,9.0,7.0,6,8,6,7,6,...,9.0,8,9.0,1,30,1,0,0,0,0
1,11.0,FAVE,7.0,7.0,8.0,8,8,8,7,7,...,7.0,8,6.0,0,29,1,0,0,0,0
2,15.0,FAVE,6.0,6.0,7.0,7,7,8,7,6,...,7.0,7,8.0,0,42,1,0,0,0,0


## Modelación

Vamos a empezar con un modelo para Amazon

In [97]:
amazon = datos2['company_v']=='AMAZON'
amazon =datos2[amazon]

(159, 26)

### Entrenamiento del modelo

In [99]:
x = amazon[amazon.columns[3:]].values
y = amazon['satis'].values

In [100]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### Crear un modelo de regresión multilineallineal

In [101]:
modelo = linear_model.LinearRegression().fit(X_train, y_train)
y_esti= modelo.predict(X_test)

### Validación del modelo

In [102]:
r2 = r2_score(y_test, y_esti)
print(f"Coeficiente de determinación R2: {r2:.2f}")

Coeficiente de determinación R2: 0.87


### calcular los errores del modelo

In [103]:
print('Error promedio - Regresion Multilineal:', round(mean_absolute_error(y_test, y_esti),3))
print('Error MSE - Regresion Multilineal:', round(mean_squared_error(y_test, y_esti),3))

Error promedio - Regresion Multilineal: 0.387
Error MSE - Regresion Multilineal: 0.251


In [104]:
colum = datos2.columns[3:]
for i in range(0,23):
    print(f'coeficiente:{colum[i]}:',round(modelo.coef_[i],3))

coeficiente:poverq: 0.096
coeficiente:soverq: 0.067
coeficiente:pq: 0.076
coeficiente:VN_1009_Q20A: 0.618
coeficiente:VN_1009_TP01: -0.003
coeficiente:VN_1009_TP02: 0.002
coeficiente:VN_1009_TP03: -0.063
coeficiente:VN_1009_TP04: 0.024
coeficiente:VN_1009_TP05: 0.117
coeficiente:VN_1009_TP06: 0.057
coeficiente:VN_1009_TP11: 0.071
coeficiente:VN_1009_TP12: 0.022
coeficiente:VN_1009_TP13: 0.077
coeficiente:VN_1009_TP16: -0.012
coeficiente:VN_1009_TP17: -0.016
coeficiente:VN_1009_TP19: 0.003
coeficiente:VN_1009_TP25A: 0.069
coeficiente:age: 0.003
coeficiente:Chinese: 0.055
coeficiente:Eurasian: 0.0
coeficiente:Indian: -0.024
coeficiente:Malay: 0.082
coeficiente:Others: -0.113
